In [ ]:

# Análisis comparativo de títulos de películas en Netflix y Amazon Prime

import pandas as pd
import numpy as np
imdb = pd.read_csv("imdb_movies.csv")
print(imdb.columns)

Index(['names', 'date_x', 'score', 'genre', 'overview', 'crew', 'orig_title',
       'status', 'orig_lang', 'budget_x', 'revenue', 'country'],
      dtype='object')


In [ ]:

# Cargar IMDb
imdb = pd.read_csv("imdb_movies.csv")
# Nos quedamos solo con columnas relevantes
imdb_clean = imdb[["names", "genre", "score", "budget_x", "revenue"]].dropna(subset=["names"])
# Normalizar títulos para comparación
imdb_clean["title_norm"] = imdb_clean["names"].str.strip().str.upper()

# Cargar datasets de streaming
amazon = pd.read_csv("amazon_prime_titles.csv")
netflix = pd.read_csv("netflix_titles.csv")
# Agregar columna de plataforma
amazon["platform"] = "Amazon Prime"
netflix["platform"] = "Netflix"
# Unir datasets
streaming = pd.concat([amazon, netflix], ignore_index=True)
# Filtrar películas con título válido
streaming = streaming[
    (streaming["type"] == "Movie") &
    (streaming["title"].notna())
].copy()
# Normalizar títulos
streaming["title_norm"] = streaming["title"].str.strip().str.upper()



In [21]:
# Filtrar solo películas que también estén en IMDb
streaming["is_in_imdb"] = streaming["title_norm"].isin(imdb_clean["title_norm"])
streaming_imdb = streaming[streaming["is_in_imdb"]].copy()

# Agrupar por título y plataforma
platforms = streaming_imdb.groupby(["title_norm", "platform"]).size().unstack(fill_value=0)

# Clasificación de disponibilidad
def classify(row):
    if row.get("Netflix", 0) > 0 and row.get("Amazon Prime", 0) > 0:
        return "Both"
    elif row.get("Netflix", 0) > 0:
        return "Netflix"
    elif row.get("Amazon Prime", 0) > 0:
        return "Amazon Prime"
    return "Unknown"

platforms["availability"] = platforms.apply(classify, axis=1)

# Juntar con IMDb
final_df = pd.merge(imdb_clean, platforms["availability"], on="title_norm", how="inner")

# Reorganizar columnas
final_df = final_df[["names", "genre", "score", "budget_x", "revenue", "availability"]]


In [22]:
# Exportar a archivo Excel
final_df.to_excel("peliculas_imdb_vs_streaming.xlsx", index=False)